In [1]:
import pandas as pd 
import numpy as np

In [2]:
from PyPDF2 import PdfReader

Importing the resume text

In [3]:
resume_file="Anmol_gupta_iitk_bt_mse.2d5042bb2dc25c3635e5.pdf"
Req="data scientist"

In [4]:
reader = PdfReader(resume_file)
page = reader.pages[0]
transcript  = page.extract_text()
print(transcript)

Anmol Gupta
4th Year Undergraduate agupta20@iitk.ac.in |/ne+91-8982511209
Department of Material Science and Engineering /gtbAnmolGupta |/♀nednanmolgupta
Academic Qualifications
Year Degree/Certificate Institute CPI/ %
2020 - Present B.Tech Indian Institute of Technology Kanpur 7.9/10
2020 Class XII (CBSE) Shanskar Public School, Gwalior 90.4%
2018 Class X (CBSE) Christ jyoti School, Lavkush nagar 90.8%
Scholastic Achievement
•Secured 9317 rank in Jee Advanced 2020
•Awarded the MCM scholarship amongst a batch of 1150+ students for the academic year 2021-22 as well as for 2022-23
Work Experience / Key Projects
Microsoft Engage |Facial Recognition Attendance System May’22
•Received extensive group mentoring from Microsoft, which helps me in enhancing my efficiency to do effective work
•Application takes all the student data including photo and store in the database of MySQL. Attendance is marked automatically
•Few games to enjoy -(1)Making the sketch automatically (2)Facial emotion using

In [5]:
job_descr = """We are looking for an Indian Institute of Technology Kanpur Grad with skills in artificial intelligence. We are looking for game development projects and other projects such as To Do lists"""

cleaning the resume text

In [6]:
description_cleaned=job_descr.replace("\n","")
resume_cleaned=transcript.replace("\n","")

# Approach 1 - Using Count Vectorizer

In [7]:
Match_Test=[resume_cleaned,description_cleaned]

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()
count_matrix=cv.fit_transform(Match_Test)

In [20]:
from sklearn.metrics.pairwise import cosine_similarity

In [21]:
MatchPercentage=cosine_similarity(count_matrix)[0][1]*100
MatchPercentage=round(MatchPercentage,2)
print('Match Percentage is :'+ str(MatchPercentage)+'% to Requirement')

Match Percentage is :38.2% to Requirement


# Approach 2 - Using TF-IDF to generate embeddings, then using cosine similarity to generate a score

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


# Create TF-IDF vectorizer
vectorizer = TfidfVectorizer(ngram_range=(3,3))

# Fit and transform the documents
tfidf_matrix = vectorizer.fit_transform([resume_cleaned, description_cleaned])

# Calculate cosine similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)


In [12]:
from sentence_transformers import SentenceTransformer

def text_similarity_bert(text1, text2):
    model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
    embeddings = model.encode([text1, text2])
    similarity_score = cosine_similarity([embeddings[0]], [embeddings[1]])
    return similarity_score[0, 0]

similarity = text_similarity_bert(resume_cleaned, description_cleaned)
print(f"Similarity Score (BERT): {similarity}")


Similarity Score (BERT): 0.41582170128822327


# Approach 3  -  Auto-FAISS to generate a similarity score - did not work as expected

In [14]:
import faiss
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize

def text_similarity_faiss(text1, text2):
    # Step 1: Vectorize the texts using TF-IDF
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform([text1, text2]).toarray()

    # Step 2: Normalize the vectors
    vectors = normalize(vectors, norm='l2')

    # Step 3: Initialize FAISS index
    index = faiss.IndexFlatL2(vectors.shape[1])
    index.add(vectors.astype(np.float32))

    # Step 4: Encode and normalize the query vector
    query_vector = vectorizer.transform([text1]).toarray()
    query_vector = normalize(query_vector, norm='l2').astype(np.float32)

    # Step 5: Perform similarity search
    _, similarity_indices = index.search(query_vector, k=1)

    # Step 6: Get similarity score
    similarity_score = 1.0 / (1.0 + similarity_indices[0, 0])

    return similarity_score

# Example usage
text1 = resume_cleaned
text2 = "123"

similarity = text_similarity_faiss(text1, text2)
print(f"Similarity Score (FAISS): {similarity}")


Similarity Score (FAISS): 1.0


# Approach 4 -  Using InstructXL retriever model to generate embeddings and using cosine similarity

Our best approach yet - generates the most accurate scores but has high inference time on our limited performance machines

In [ ]:
from InstructorEmbedding import INSTRUCTOR
model = INSTRUCTOR('hkunlp/instructor-xl')
sentence = "3D ActionSLAM: wearable person tracking in multi-floor environments"
instruction = "Represent the Science title:"
embeddings = model.encode([[instruction,sentence]])

In [16]:
sentence = resume_cleaned
instruction = "extract skills and experience from the resume"
resume_embeddings = model.encode([[instruction,sentence]])


In [17]:
sentence = description_cleaned
instruction = "extract requirements and candidate skills"
description_embeddings = model.encode([[instruction,sentence]])


In [18]:
similarity_score = cosine_similarity(resume_embeddings, description_embeddings)
print(((similarity_score[0][0]*100) - 50)/(50) * 100)

67.62865781784058


In [19]:
similarity_score

array([[0.8381433]], dtype=float32)